In [ ]:
import os
import shutil
import pandas as pd
from sklearn.model_selection import train_test_split
from PIL import Image

# Paths
input_dir =  r"C:\Users\Admin\JupyterLab\FloodDetectionProject\devset_images\devset_images"
output_base_dir = r"C:\Users\Admin\JupyterLab\FloodDetectionProject\data"
csv_file = r"C:\Users\Admin\JupyterLab\FloodDetectionProject\devset_images_gt.csv"

# Directory structure
dirs = {
    'train': ['Flooding', 'No_Flooding'],
    'valid': ['Flooding', 'No_Flooding'],
    'test': ['Flooding', 'No_Flooding']
}

# Create the directory structure
for split in dirs:
    for category in dirs[split]:
        os.makedirs(os.path.join(output_base_dir, split, category), exist_ok=True)

# Read the CSV file
labels_df = pd.read_csv(csv_file)

# Ensure filenames end with '.jpg'
labels_df['id'] = labels_df['id'].apply(lambda x: x if str(x).lower().endswith('.jpg') else str(x) + '.jpg')

# Split the data
train_df, temp_df = train_test_split(labels_df, test_size=0.4, stratify=labels_df['label'], random_state=42)
valid_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['label'], random_state=42)

# Function to move images to the correct directory
def move_images(df, split):
    for index, row in df.iterrows():
        filename = row['id']
        label = row['label']
        category = 'Flooding' if label == 1 else 'No_Flooding'
        
        # Construct the full path to the input and output images
        src_path = os.path.join(input_dir, filename)
        dst_path = os.path.join(output_base_dir, split, category, filename)
        
        # Move the image
        try:
            shutil.copy(src_path, dst_path)
        except FileNotFoundError:
            print(f"File not found: {src_path}")
        except Exception as e:
            print(f"Error copying file {src_path} to {dst_path}: {e}")

# Move images to their respective directories
move_images(train_df, 'train')
move_images(valid_df, 'valid')
move_images(test_df, 'test')

print("Images have been categorized and moved to the appropriate directories.")
